In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
path = "./data/Imports_Exports_Dataset.csv"
data = spark.read.option("header", True).csv(path)

In [4]:
data.createOrReplaceTempView("Transactions")

In [5]:
data.show(5)

+--------------------+----------------+---------+-------------+--------+-------+----------+-----------+----------------+------------+-------+---------------+--------------------+-------------+--------------+----------------+
|      Transaction_ID|         Country|  Product|Import_Export|Quantity|  Value|      Date|   Category|            Port|Customs_Code| Weight|Shipping_Method|            Supplier|     Customer|Invoice_Number|   Payment_Terms|
+--------------------+----------------+---------+-------------+--------+-------+----------+-----------+----------------+------------+-------+---------------+--------------------+-------------+--------------+----------------+
|e3e70682-c209-4ca...|        Colombia| describe|       Export|    1979|9506.57|07-12-2023|  Machinery|      Robertbury|      620537|4248.65|            Air|Garrison, Hubbard...|    Seth Hall|      21000294|Cash on Delivery|
|f728b4fa-4248-4e3...|           Chile|president|       Export|    5763|7100.91|04-04-2023|   Clothi

In [6]:
query = "SELECT DISTINCT Country FROM Transactions"
Country_Dim = spark.sql(query)

In [7]:
Country_Dim.show(5)

+--------------------+
|             Country|
+--------------------+
|                Chad|
|            Paraguay|
|            Anguilla|
|               Macao|
|Heard Island and ...|
+--------------------+
only showing top 5 rows



In [8]:
Country_Dim = Country_Dim.withColumn("Country_ID", row_number().over(Window.orderBy(Country_Dim.Country)))

In [9]:
Country_Dim = Country_Dim.withColumnRenamed("Country", "Country_Name")
Country_Dim.show(10)

+--------------------+----------+
|        Country_Name|Country_ID|
+--------------------+----------+
|         Afghanistan|         1|
|             Albania|         2|
|             Algeria|         3|
|      American Samoa|         4|
|             Andorra|         5|
|              Angola|         6|
|            Anguilla|         7|
|Antarctica (the t...|         8|
| Antigua and Barbuda|         9|
|           Argentina|        10|
+--------------------+----------+
only showing top 10 rows



In [10]:
query = "SELECT DISTINCT Product FROM Transactions"
Product_Dim = spark.sql(query)

In [11]:
Product_Dim.show(5)

+---------+
|  Product|
+---------+
|    those|
|      few|
|     hope|
|recognize|
|     some|
+---------+
only showing top 5 rows



In [12]:
Product_Dim = Product_Dim.withColumn("Product_ID", row_number().over(Window.orderBy(Product_Dim.Product)))

In [13]:
Product_Dim = Product_Dim.withColumnRenamed("Product", "Product_Name")
Product_Dim.show(10)

+------------+----------+
|Product_Name|Product_ID|
+------------+----------+
|    American|         1|
|    Congress|         2|
|    Democrat|         3|
|           I|         4|
|          Mr|         5|
|         Mrs|         6|
|          PM|         7|
|  Republican|         8|
|        TRUE|         9|
|          TV|        10|
+------------+----------+
only showing top 10 rows



In [14]:
query = "SELECT DISTINCT Category FROM Transactions"
Category_Dim = spark.sql(query)

In [15]:
Category_Dim.show(5)

+-----------+
|   Category|
+-----------+
|  Machinery|
|Electronics|
|   Clothing|
|  Furniture|
|       Toys|
+-----------+



In [16]:
Category_Dim = Category_Dim.withColumn("Category_ID", row_number().over(Window.orderBy(Category_Dim.Category)))

In [17]:
Category_Dim = Category_Dim.withColumnRenamed("Category", "Category_Name")
Category_Dim.show(10)

+-------------+-----------+
|Category_Name|Category_ID|
+-------------+-----------+
|     Clothing|          1|
|  Electronics|          2|
|    Furniture|          3|
|    Machinery|          4|
|         Toys|          5|
+-------------+-----------+



In [18]:
query = "SELECT DISTINCT Supplier FROM Transactions"
Supplier_Dim = spark.sql(query)

In [19]:
Supplier_Dim.show(5)

+--------------------+
|            Supplier|
+--------------------+
|           Smith PLC|
|Smith, Price and ...|
|Jennings, Thomas ...|
|        Gibson Group|
|         Casey-Evans|
+--------------------+
only showing top 5 rows



In [20]:
Supplier_Dim = Supplier_Dim.withColumn("Supplier_ID", row_number().over(Window.orderBy(Supplier_Dim.Supplier)))

In [21]:
Supplier_Dim = Supplier_Dim.withColumnRenamed("Supplier", "Supplier_Name")
Supplier_Dim.show(10)

+--------------------+-----------+
|       Supplier_Name|Supplier_ID|
+--------------------+-----------+
|          Abbott LLC|          1|
|     Abbott and Sons|          2|
|Abbott, Olson and...|          3|
|Abbott, Taylor an...|          4|
|     Abbott-Oconnell|          5|
|        Abbott-Patel|          6|
|         Acevedo LLC|          7|
|Acevedo, Atkins a...|          8|
|Acevedo, Gonzales...|          9|
|Acevedo, Ortiz an...|         10|
+--------------------+-----------+
only showing top 10 rows



In [22]:
query = "SELECT DISTINCT Customer FROM Transactions"
Customer_Dim = spark.sql(query)

In [23]:
Customer_Dim.show(5)

+-------------+
|     Customer|
+-------------+
|  Kyle Nelson|
|Jeremy Hunter|
| Jasmine Long|
| Ronald Hayes|
|    John Dean|
+-------------+
only showing top 5 rows



In [24]:
Customer_Dim = Customer_Dim.withColumn("Customer_ID", row_number().over(Window.orderBy(Customer_Dim.Customer)))

In [25]:
Customer_Dim = Customer_Dim.withColumnRenamed("Customer", "Customer_Name")
Customer_Dim.show(10)

+-----------------+-----------+
|    Customer_Name|Customer_ID|
+-----------------+-----------+
|      Aaron Ayers|          1|
|      Aaron Baker|          2|
|      Aaron Banks|          3|
|     Aaron Barnes|          4|
|      Aaron Brown|          5|
|     Aaron Butler|          6|
|Aaron Carroll Jr.|          7|
|  Aaron Castaneda|          8|
|     Aaron Chaney|          9|
|      Aaron Cline|         10|
+-----------------+-----------+
only showing top 10 rows



In [28]:
data = data.join(Country_Dim, data["Country"] == Country_Dim["Country_Name"])
data = data.join(Category_Dim, data["Category"] == Category_Dim["Category_Name"])
data = data.join(Product_Dim, data["Product"] == Product_Dim["Product_Name"])
data = data.join(Supplier_Dim, data["Supplier"] == Supplier_Dim["Supplier_Name"])
data = data.join(Customer_Dim, data["Customer"] == Customer_Dim["Customer_Name"])

In [29]:
Transactions_Fact = data.withColumn("ID", row_number().over(Window.orderBy(data.Transaction_ID)))
Transactions_Fact = Transactions_Fact.withColumn("Transaction_Type", Transactions_Fact["Import_Export"])
Transactions_Fact = Transactions_Fact[["ID","Transaction_ID","Country_ID", "Product_ID", "Transaction_Type","Quantity", "Value", "Date", "Category_ID", "Port", "Customs_Code", "Weight", "Shipping_Method", "Supplier_ID", "Customer_ID", "Invoice_Number", "Payment_Terms"]]

Transactions_Fact.show(5)

+---+--------------------+----------+----------+----------------+--------+-------+----------+-----------+-----------+------------+-------+---------------+-----------+-----------+--------------+-------------+
| ID|      Transaction_ID|Country_ID|Product_ID|Transaction_Type|Quantity|  Value|      Date|Category_ID|       Port|Customs_Code| Weight|Shipping_Method|Supplier_ID|Customer_ID|Invoice_Number|Payment_Terms|
+---+--------------------+----------+----------+----------------+--------+-------+----------+-----------+-----------+------------+-------+---------------+-----------+-----------+--------------+-------------+
|  1|00073cc2-c801-467...|        77|       866|          Export|    2434|9753.17|30-09-2023|          5|  Davisbury|      194680|4111.53|           Land|       7387|       1530|      32236195|      Prepaid|
|  2|000b6710-f8b4-4dd...|        83|       675|          Export|    7036|  728.5|25-06-2020|          4|Russellport|      492898|4893.37|            Sea|       1845|  

In [ ]:
Transactions_Fact.toPandas().to_csv("./output/Transaction_Fact.csv")
Country_Dim.toPandas().to_csv("./output/Country_Dim.csv")
Category_Dim.toPandas().to_csv("./output/Category_Dim.csv")
Product_Dim.toPandas().to_csv("./output/Product_Dim.csv")
Supplier_Dim.toPandas().to_csv("./output/Supplier_Dim.csv")
Customer_Dim.toPandas().to_csv("./output/Customer_Dim.csv")